In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer

In [2]:
spotify_data = pd.read_csv('./SpotifyAudioFeaturesApril2019.csv')

/home/akshat/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
spotify_data.shape

(42305, 22)

In [4]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      42305 non-null  float64
 1   energy            42305 non-null  float64
 2   key               42305 non-null  int64  
 3   loudness          42305 non-null  float64
 4   mode              42305 non-null  int64  
 5   speechiness       42305 non-null  float64
 6   acousticness      42305 non-null  float64
 7   instrumentalness  42305 non-null  float64
 8   liveness          42305 non-null  float64
 9   valence           42305 non-null  float64
 10  tempo             42305 non-null  float64
 11  type              42305 non-null  object 
 12  id                42305 non-null  object 
 13  uri               42305 non-null  object 
 14  track_href        42305 non-null  object 
 15  analysis_url      42305 non-null  object 
 16  duration_ms       42305 non-null  int64 

In [5]:
song_name = spotify_data["song_name"]

In [6]:
print(song_name.shape)
print(song_name.isnull().values.any())

(42305,)
True


In [7]:
song_name = song_name.values.reshape(-1,1)

In [8]:
song_name.shape

(42305, 1)

In [9]:
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(song_name)
imputed_data = imr.transform(song_name)
song_name = pd.DataFrame(imputed_data)
song_name = song_name.rename(columns={0:"Song-Names"})
song_name.head()

,Song-Names
0,Mercury: Retrograde
1,Pathology
2,Symbiote
3,ProductOfDrugs (Prod. The Virus and Antidote)
4,Venom


In [10]:
df = spotify_data[["genre","mode","duration_ms"]]
print(df.dtypes)
print(df.head())

genre          object
mode            int64
duration_ms     int64
dtype: object
       genre  mode  duration_ms
0  Dark Trap     1       124539
1  Dark Trap     1       224427
2  Dark Trap     1        98821
3  Dark Trap     1       123661
4  Dark Trap     1       123298


In [11]:
df.dtypes

genre          object
mode            int64
duration_ms     int64
dtype: object

In [12]:
df["genre"].value_counts()

Underground Rap    5875
Dark Trap          4578
Hiphop             3028
trance             2999
trap               2987
techhouse          2975
dnb                2966
psytrance          2961
techno             2956
hardstyle          2936
RnB                2099
Trap Metal         1956
Rap                1848
Emo                1680
Pop                 461
Name: genre, dtype: int64

In [13]:
df = df.replace({"genre":{"Underground Rap":0, "Dark Trap":1, "Hiphop":2, "trance":3, "trap":4, "techhouse":5, "dnb":6, "psytrance": 7, "techno":8, "hardstyle":9, "RnB":10, "Trap Metal":11, "Rap":12, "Emo":13, "Pop":14}})

In [14]:
nmf = NMF(n_components = 6)
nmf_features = nmf.fit_transform(df)

/home/akshat/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [15]:
norm_features = normalize(nmf_features)
current_music = norm_features[23,:]
similarities = norm_features.dot(current_music)
similarities

array([9.96314415e-01, 9.93855206e-01, 1.97643638e-09, ...,
       9.73386535e-01, 9.57998355e-01, 9.88694857e-01])

In [16]:
df2 = pd.DataFrame(norm_features)
x = df2.join(song_name)
df2 = pd.pivot_table(x, x[[0,1,2,3,4,5]],["Song-Names"])
def current_music(value):
    print("Top 5 recommendations for given music are:")
    value = df2.loc[value]
    similarities = df2.dot(value)
    print(format(similarities.nlargest()))

In [17]:
current_music("Venom")

Top 5 recommendations for given music are:
Song-Names
Touch My Body                                0.961032
Slide (with Gunna)                           0.960898
Make Tonight                                 0.960407
Secret Admirer                               0.960227
Good Lovin (feat. Fabolous & Ryan Leslie)    0.960226
dtype: float64
